In [1]:
import deep_image_matching as dim

params = {
    "dir": "./assets/example_cyprus",
    "pipeline": "superpoint+lightglue",
    "strategy": "matching_lowres",
    "quality": "medium",
    "tiling": "none",
    "camera_options": "./assets/example_cyprus/cameras.yaml",
    "openmvg": None,
}
config = dim.Config(params)

Deep Image Matching loaded in 3.014 seconds.
2024-05-13 14:08:21 | [WARNING ] assets/example_cyprus/results_superpoint+lightglue_matching_lowres_quality_medium already exists, but the '--force' option is used. Deleting the folder.


Run feature extraction and matching

In [2]:
matcher = dim.ImageMatcher(config)
feature_path, match_path = matcher.run()

Loaded SuperPoint model
2024-05-13 14:08:22 | [INFO    ] Running image matching with the following configuration:
2024-05-13 14:08:22 | [INFO    ]   Image folder: assets/example_cyprus/images
2024-05-13 14:08:22 | [INFO    ]   Output folder: assets/example_cyprus/results_superpoint+lightglue_matching_lowres_quality_medium
2024-05-13 14:08:22 | [INFO    ]   Number of images: 10
2024-05-13 14:08:22 | [INFO    ]   Matching strategy: matching_lowres
2024-05-13 14:08:22 | [INFO    ]   Image quality: MEDIUM
2024-05-13 14:08:22 | [INFO    ]   Tile selection: NONE
2024-05-13 14:08:22 | [INFO    ]   Feature extraction method: superpoint
2024-05-13 14:08:22 | [INFO    ]   Matching method: lightglue
2024-05-13 14:08:22 | [INFO    ]   Geometric verification: PYDEGENSAC
2024-05-13 14:08:22 | [INFO    ]   CUDA available: True
2024-05-13 14:08:22 | [INFO    ] Low resolution matching, generating pairs ..
Loaded SuperPoint model
2024-05-13 14:08:23 | [INFO    ] Extracting features from downsampled imag

  0%|          | 0/10 [00:00<?, ?it/s]/home/francesco/miniforge3/envs/dim/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
100%|██████████| 10/10 [00:02<00:00,  3.47it/s]

2024-05-13 14:08:25 | [INFO    ] Matching downsampled images...



100%|██████████| 45/45 [00:00<00:00, 49.99it/s]

2024-05-13 14:08:26 | [INFO    ] Found 28 pairs.
2024-05-13 14:08:26 | [INFO    ] Extracting features with superpoint...
2024-05-13 14:08:26 | [INFO    ] superpoint configuration: 


{'keypoint_threshold': 0.0005,
 'max_keypoints': 4096,
 'name': 'superpoint',
 'nms_radius': 3}


100%|██████████| 10/10 [00:00<00:00, 11.16it/s]

2024-05-13 14:08:27 | [INFO    ] Features extracted!
2024-05-13 14:08:27 | [INFO    ] Matching features with lightglue...
2024-05-13 14:08:27 | [INFO    ] lightglue configuration: 
{'depth_confidence': 0.95,
 'filter_threshold': 0.1,
 'flash': True,
 'mp': False,
 'n_layers': 9,
 'name': 'lightglue',
 'width_confidence': 0.99}
2024-05-13 14:08:27 | [INFO    ] Matching features...
2024-05-13 14:08:27 | [INFO    ] 



100%|██████████| 28/28 [00:09<00:00,  3.04it/s]

2024-05-13 14:08:37 | [INFO    ] [Timer] | [matching] generate_pairs=5.171, extract_features=0.905, Match pair=0.370, Total execution=15.308
2024-05-13 14:08:37 | [INFO    ] [Timer] | [Deep Image Matching] Total execution=0.000


Export matches to COLMAP format

In [3]:
dim.io.export_to_colmap(
    img_dir=config.general["image_dir"],
    feature_path=feature_path,
    match_path=match_path,
    database_path=config.general["output_dir"] / "database.db",
    camera_config_path=config.general["camera_options"],
)

2024-05-13 14:08:37 | [WARNING ] Was not possible to load the first image to initialize cam0
2024-05-13 14:08:37 | [WARNING ] Was not possible to load the first image to initialize cam1


100%|██████████| 10/10 [00:00<00:00, 693.76it/s]
28it [00:00, 6981.78it/s]             
28it [00:00, 7524.38it/s]             


Run reconstruction with pycolmap

In [4]:
model = dim.reconstruction.pycolmap_reconstruction(
    database_path=config.general["output_dir"] / "database.db",
    sfm_dir=config.general["output_dir"],
    image_dir=config.general["image_dir"],
    refine_intrinsics=False,
)

2024-05-13 14:08:37 | [INFO    ] Running 3D reconstruction...


I20240513 14:08:37.102010 3263722 misc.cc:198] 
Loading database
I20240513 14:08:37.103212 3263722 database_cache.cc:54] Loading cameras...
I20240513 14:08:37.103258 3263722 database_cache.cc:64]  1 in 0.000s
I20240513 14:08:37.103271 3263722 database_cache.cc:72] Loading matches...
I20240513 14:08:37.103843 3263722 database_cache.cc:78]  28 in 0.001s
I20240513 14:08:37.103858 3263722 database_cache.cc:94] Loading images...
I20240513 14:08:37.104916 3263722 database_cache.cc:143]  10 in 0.001s (connected 8)
I20240513 14:08:37.104933 3263722 database_cache.cc:154] Building correspondence graph...
I20240513 14:08:37.109264 3263722 database_cache.cc:190]  in 0.004s (ignored 0)
I20240513 14:08:37.109319 3263722 timer.cc:91] Elapsed time: 0.000 [minutes]
I20240513 14:08:37.109831 3263722 misc.cc:198] 
Finding good initial image pair
I20240513 14:08:37.262360 3263722 misc.cc:198] 
Initializing with image pair #5 and #9
I20240513 14:08:37.265554 3263722 misc.cc:198] 
Global bundle adjustment


2024-05-13 14:08:42 | [INFO    ] Reconstructed 1 model(s).
2024-05-13 14:08:42 | [INFO    ] Largest model is #0 with 8 images.
2024-05-13 14:08:42 | [INFO    ] Exporting model...


I20240513 14:08:41.976262 3263722 misc.cc:205] 
Bundle adjustment report
------------------------
I20240513 14:08:41.976303 3263722 bundle_adjustment.cc:942] 
    Residuals : 24806
   Parameters : 12653
   Iterations : 5
         Time : 0.0700741 [s]
 Initial cost : 0.814227 [px]
   Final cost : 0.811154 [px]
  Termination : Convergence

I20240513 14:08:41.981084 3263722 incremental_mapper.cc:175] => Completed observations: 14
I20240513 14:08:41.982640 3263722 incremental_mapper.cc:178] => Merged observations: 0
I20240513 14:08:41.983677 3263722 incremental_mapper.cc:160] => Filtered observations: 12
I20240513 14:08:41.983688 3263722 incremental_mapper.cc:119] => Changed observations: 0.002096
I20240513 14:08:41.983697 3263722 incremental_mapper.cc:167] => Filtered images: 0
I20240513 14:08:42.012581 3263722 timer.cc:91] Elapsed time: 0.082 [minutes]
